In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torchvision.models as models
import numpy as np
import pandas as pd
import random
import wandb
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")

wandb_api_key = secret_value_0 #os.getenv("WANDB_API_KEY")
wandb.login(key=wandb_api_key)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m021 (da24m021-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### Prepare Data

In [4]:
TRAIN_FilePath = 'dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv'
DEV_FilePath = 'dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv'
TEST_FilePath = 'dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv'

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')
TRAIN_FilePath = "/kaggle/input/dakshinahindi/hi/lexicons/hi.translit.sampled.train.tsv"
DEV_FilePath = "/kaggle/input/dakshinahindi/hi/lexicons/hi.translit.sampled.dev.tsv"
TEST_FilePath = "/kaggle/input/dakshinahindi/hi/lexicons/hi.translit.sampled.test.tsv"

In [6]:
# Load train, dev and test datasets
train_df = pd.read_csv(TRAIN_FilePath, sep='\t', header=None)
dev_df = pd.read_csv(DEV_FilePath, sep='\t', header=None)
test_df = pd.read_csv(TEST_FilePath, sep='\t', header=None)


# Renaming the columns
train_df.columns = ['devanagari', 'latin', 'frequency']
dev_df.columns = ['devanagari', 'latin', 'frequency']
test_df.columns = ['devanagari', 'latin', 'frequency']

# Dataset Sizes
print(f"Train Dataset Size : {train_df.shape[0]}\nDev Dataset Size   : {dev_df.shape[0]}\nTest Dataset Size  : {test_df.shape[0]}")

Train Dataset Size : 44204
Dev Dataset Size   : 4358
Test Dataset Size  : 4502


In [7]:
train_df.head()

,devanagari,latin,frequency
0,अं,an,3
1,अंकगणित,ankganit,3
2,अंकल,uncle,4
3,अंकुर,ankur,4
4,अंकुरण,ankuran,3


In [8]:
class Vocabulary:
    def __init__(self):
        self.pad_token = "<pad>"
        self.sos_token = "<sos>"
        self.eos_token = "<eos>"
        self.unk_token = "<unk>"

        # Initialize mappings
        self.char2idx = {self.pad_token: 0, self.sos_token: 1, self.eos_token: 2, self.unk_token: 3}
        self.idx2char = {0: self.pad_token, 1: self.sos_token, 2: self.eos_token, 3: self.unk_token}
        self.vocab_size = 4

    def build_vocabulary(self, text_data):
        for text in text_data:
            text = str(text)
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.vocab_size
                    self.idx2char[self.vocab_size] = char
                    self.vocab_size += 1

    def encode(self, text, add_special_tokens=True):
        indices = []
        text = str(text)
        for char in text:
            indices.append(self.char2idx.get(char, self.char2idx[self.unk_token]))

        if add_special_tokens:
            indices = [self.char2idx[self.sos_token]] + indices + [self.char2idx[self.eos_token]]

        return indices

    def decode(self, indices, remove_special_tokens=True):
        chars = []
        keys = list(self.idx2char.keys())
        for idx in indices:
            if isinstance(idx, torch.Tensor):
                idx = idx.item()
            if idx in keys:
                char = self.idx2char[idx]
                if remove_special_tokens and char in [self.pad_token, self.sos_token, self.eos_token, self.unk_token]:
                    continue
                chars.append(char)

        return "".join(chars)


In [9]:
class TransliterationDataset(Dataset):
    def __init__(self, data_path, src_vocab, tgt_vocab):
        df = pd.read_csv(data_path, sep='\t', header=None)

        # Create Dataset
        self.source_sequences = []
        self.target_sequences = []

        for idx, row in df.iterrows():
            x_seq = src_vocab.encode(row[1])
            y_seq = tgt_vocab.encode(row[0])
            self.source_sequences.append(x_seq)
            self.target_sequences.append(y_seq)

    def __len__(self):
        return len(self.source_sequences)

    def __getitem__(self, idx):
        return torch.tensor(self.source_sequences[idx], dtype=torch.long), torch.tensor(self.target_sequences[idx], dtype=torch.long)


In [10]:
def collate_fn(batch):
    src_batch = [item[0] for item in batch]
    tgt_batch = [item[1] for item in batch]

    # Pad sequences
    src_batch_padded = pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_batch_padded = pad_sequence(tgt_batch, batch_first=True, padding_value=0)

    return src_batch_padded, tgt_batch_padded

In [11]:
# Build Source and Target Vocabularies
src_vocab = Vocabulary()
tgt_vocab = Vocabulary()

train_df = pd.read_csv(TRAIN_FilePath, sep='\t', header=None)
src_text = []
tgt_text = []
for idx, row in train_df.iterrows():
    src_text.append(row[1])
    tgt_text.append(row[0])

src_vocab.build_vocabulary(src_text)
tgt_vocab.build_vocabulary(tgt_text)

In [12]:
# Train, Test and Dev (Validation) Dataset and Dataloaders
train_dataset = TransliterationDataset(TRAIN_FilePath, src_vocab, tgt_vocab)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

dev_dataset = TransliterationDataset(DEV_FilePath, src_vocab, tgt_vocab)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

test_dataset = TransliterationDataset(TEST_FilePath, src_vocab, tgt_vocab)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


In [13]:
# Inspect if dataloader is created as desired
for batch in train_loader:
    inputs, targets = batch

    print("Inputs shape:", inputs.shape)
    print("Targets shape:", targets.shape)

    # Check one sample
    print("Sample input:", inputs[0])
    print("Sample target:", targets[0])
    print(src_vocab.decode(inputs[0]))
    print(tgt_vocab.decode(targets[0]))
    break

Inputs shape: torch.Size([32, 15])
Targets shape: torch.Size([32, 13])
Sample input: tensor([ 1, 23, 10, 16,  4, 14, 20,  5,  2,  0,  0,  0,  0,  0,  0])
Sample target: tensor([ 1, 29, 12, 34, 13, 25,  5,  2,  0,  0,  0,  0,  0])
muharon
मुहरों


## Model

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers=1, cell_type="RNN", dropout=0.0):
        super().__init__()

        if num_layers == 1:
            dropout = 0

        self.cell_type = cell_type

        # Embedding Layer
        self.embedding = nn.Embedding(input_size, embedding_size)

        # Recurrent Layer
        if cell_type == "LSTM":
            self.recurrent_layer = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        elif cell_type == "GRU":
            self.recurrent_layer = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        else: # Default (RNN)
            self.recurrent_layer = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        # input: batch_size x seq_len

        embeddings = self.embedding(input)
        embeddings = self.dropout(embeddings)

        if self.cell_type == 'LSTM':
            outputs, (hidden, cell) = self.recurrent_layer(embeddings)
            return outputs, (hidden, cell)
        else:
            outputs, hidden = self.recurrent_layer(embeddings)
            return outputs, hidden


In [15]:
class Decoder(nn.Module):
    def __init__(self, output_size, embedding_size, hidden_size, num_layers=1, cell_type="RNN", dropout=0.0):
        super().__init__()

        if num_layers == 1:
            dropout = 0

        self.output_size = output_size
        self.cell_type = cell_type

        # Embedding Layer
        self.embedding = nn.Embedding(output_size, embedding_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        # Recurrent Layer
        if cell_type == "LSTM":
            self.recurrent_layer = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        elif cell_type == "GRU":
            self.recurrent_layer = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        else: # Default (RNN)
            self.recurrent_layer = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)

        # Output layer
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):

        input = input.unsqueeze(1)
        embeddings = self.embedding(input)
        embeddings = self.dropout(embeddings)

        if self.cell_type == "LSTM":
            hidden, cell = hidden
            outputs, (hidden, cell) = self.recurrent_layer(embeddings, (hidden, cell))
            hidden = (hidden, cell)
        else:
            outputs, hidden = self.recurrent_layer(embeddings, hidden)

        outputs = outputs.squeeze(1)
        prediction = self.fc_out(outputs)
        return prediction, hidden


In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):

        batch_size = src.shape[0]
        tgt_len = tgt.shape[1]
        tgt_vocab_size = self.decoder.output_size

        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        if self.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = self.encoder(src)
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = self.encoder(src)
            decoder_hidden = hidden

        decoder_input = tgt[:, 0]
        for t in range(1, tgt_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[:, t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio
            top = decoder_output.argmax(1)
            decoder_input = tgt[:, t] if teacher_force else top
        return outputs


    def inference(self, src, max_len, sos_idx=1, eos_idx=2):

        batch_size = src.shape[0]
        tgt_len = max_len
        tgt_vocab_size = self.decoder.output_size
        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        if self.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = self.encoder(src)
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = self.encoder(src)
            decoder_hidden = hidden

        decoder_input = torch.tensor([sos_idx] * batch_size, device=self.device)
        for t in range(1, max_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[:, t] = decoder_output
            top = decoder_output.argmax(1)
            decoder_input = top

            # Check if all sequences have reached <eos>
            if (outputs == eos_idx).any(dim=1).all():
                break
        return outputs

In [17]:
def train(model, train_loader, optimizer, criterion, clip=1.0):
    model.train()
    epoch_loss = 0

    for i, (src, tgt) in enumerate(train_loader):
        src = src.to(device)
        tgt = tgt.to(device)
        #print("tgt", tgt)
        optimizer.zero_grad()

        output = model(src, tgt)
        #print("output", output)

        # Reshape output and target for loss calculation
        # output: [batch_size, tgt_len, output_dim]
        # tgt: [batch_size, tgt_len]
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)  # Remove first token (SOS)
        tgt = tgt[:, 1:].reshape(-1)  # Remove first token (SOS)
        #print("tgt2", tgt)
        #print("output2", output)
        #break
        # Calculate loss
        loss = criterion(output, tgt)

        # Backpropagation
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Update parameters
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)

In [18]:
def evaluate(model, val_loader, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, (src, tgt) in enumerate(val_loader):
            src = src.to(device)
            tgt = tgt.to(device)

            output = model.inference(src, tgt.shape[1])

            # Reshape output and target for loss calculation
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            tgt = tgt[:, 1:].reshape(-1)

            # Calculate loss
            loss = criterion(output, tgt)
            epoch_loss += loss.item()

    return epoch_loss / len(val_loader)

In [19]:
def transliterate(model, src_text, src_vocab, tgt_vocab, device, max_length=100):
    model.eval()

    # Convert source text to tensor
    src_indices = src_vocab.encode(src_text)
    src_tensor = torch.tensor(src_indices, dtype=torch.long).unsqueeze(0).to(device)

    # Get encoder outputs
    with torch.no_grad():
        if model.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = model.encoder(src_tensor)
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = model.encoder(src_tensor)
            decoder_hidden = hidden

    # Start with SOS token
    decoder_input = torch.tensor([tgt_vocab.char2idx[tgt_vocab.sos_token]], device=device)

    result_indices = [tgt_vocab.char2idx[tgt_vocab.sos_token]]

    for _ in range(max_length):
        with torch.no_grad():
            decoder_output, decoder_hidden = model.decoder(decoder_input, decoder_hidden)

        # Get the most likely next character
        top_token = decoder_output.argmax(1).item()
        result_indices.append(top_token)

        # Stop if EOS token
        if top_token == tgt_vocab.char2idx[tgt_vocab.eos_token]:
            break

        # Use predicted token as next input
        decoder_input = torch.tensor([top_token], device=device)

    # Convert indices to text
    result_text = tgt_vocab.decode(result_indices)

    return result_text

In [20]:
INPUT_SIZE = src_vocab.vocab_size
OUTPUT_SIZE = tgt_vocab.vocab_size
EMBEDDING_SIZE = 256
HIDDEN_SIZE = 512
NUM_LAYERS = 2
CELL_TYPE = "LSTM"
DROPOUT = 0.2
LEARNING_RATE = 0.001
NUM_EPOCHS = 10

# Initialize encoder, decoder, and seq2seq model
encoder = Encoder(
    input_size=INPUT_SIZE,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    cell_type=CELL_TYPE,
    dropout=DROPOUT
)

decoder = Decoder(
    output_size=OUTPUT_SIZE,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    cell_type=CELL_TYPE,
    dropout=DROPOUT
)

model = Seq2Seq(encoder, decoder, device).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding token (index 0)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [21]:
# Training loop
print(f"Starting training for {NUM_EPOCHS} epochs...")

for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")

    # Train model
    train_loss = train(model, train_loader, optimizer, criterion)

    # Evaluate model
    valid_loss = evaluate(model, dev_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f}")
    print("="*20)

Starting training for 10 epochs...
Epoch 1/10
Train Loss: 1.6785 | Valid Loss: 1.1855
Epoch 2/10
Train Loss: 0.7500 | Valid Loss: 1.0837
Epoch 3/10
Train Loss: 0.5640 | Valid Loss: 1.0459
Epoch 4/10
Train Loss: 0.4627 | Valid Loss: 1.0494
Epoch 5/10
Train Loss: 0.3941 | Valid Loss: 1.0288
Epoch 6/10
Train Loss: 0.3373 | Valid Loss: 1.1191
Epoch 7/10
Train Loss: 0.2952 | Valid Loss: 1.0998
Epoch 8/10
Train Loss: 0.2599 | Valid Loss: 1.1778
Epoch 9/10
Train Loss: 0.2376 | Valid Loss: 1.2319
Epoch 10/10
Train Loss: 0.2176 | Valid Loss: 1.2525


In [22]:
# Accuracy calculation function
def calculate_accuracy(model, data_loader, src_vocab, tgt_vocab, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for src, tgt in data_loader:
            src = src.to(device)
            tgt = tgt.to(device)

            batch_size = src.shape[0]

            for i in range(batch_size):
                # Get source text and actual target text
                src_indices = src[i].tolist()
                src_text = src_vocab.decode(src_indices)
                actual_tgt_text = tgt_vocab.decode(tgt[i].tolist())

                # Get predicted transliteration
                predicted_tgt_text = transliterate(model, src_text, src_vocab, tgt_vocab, device)

                # Check if prediction matches
                if predicted_tgt_text == actual_tgt_text:
                    correct += 1
                total += 1

    return correct / total

val_accuracy = calculate_accuracy(model, dev_loader, src_vocab, tgt_vocab, device)
print(f"\nValidation Accuracy: {val_accuracy:.4f}")


Validation Accuracy: 0.3653


In [23]:
def sweep_hyperparameters(config=None):
    with wandb.init(config=config):
        config = wandb.config
        wandb.run.name = f"embedding_size_{str(config.embedding_size)}_num_layers_{str(config.num_layers)}_hidden_size_{config.hidden_size} \
                          _cell_type_{config.cell_type}_dp_{config.dropout}_lr_{config.learning_rate}_batch_{config.batch_size}"

        # Log in my details
        wandb.config.update({"NAME": "SIDDHANT BARANWAL", "ROLL NO.": "DA24M021"})

        train_dataset = TransliterationDataset(TRAIN_FilePath, src_vocab, tgt_vocab)
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, collate_fn=collate_fn)

        dev_dataset = TransliterationDataset(DEV_FilePath, src_vocab, tgt_vocab)
        dev_loader = DataLoader(dev_dataset, batch_size=config.batch_size, shuffle=False, collate_fn=collate_fn)

        INPUT_SIZE = src_vocab.vocab_size
        OUTPUT_SIZE = tgt_vocab.vocab_size

        EMBEDDING_SIZE = config.embedding_size
        HIDDEN_SIZE = config.hidden_size
        NUM_LAYERS = config.num_layers
        CELL_TYPE = config.cell_type
        DROPOUT = config.dropout
        LEARNING_RATE = config.learning_rate
        NUM_EPOCHS = 10

        # Initialize encoder, decoder, and seq2seq model
        encoder = Encoder(
            input_size=INPUT_SIZE,
            embedding_size=EMBEDDING_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LAYERS,
            cell_type=CELL_TYPE,
            dropout=DROPOUT
        )

        decoder = Decoder(
            output_size=OUTPUT_SIZE,
            embedding_size=EMBEDDING_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LAYERS,
            cell_type=CELL_TYPE,
            dropout=DROPOUT
        )

        model = Seq2Seq(encoder, decoder, device).to(device)
        criterion = nn.CrossEntropyLoss(ignore_index=0)
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

        for epoch in tqdm(range(NUM_EPOCHS)):
            # Train model
            train_loss = train(model, train_loader, optimizer, criterion)
            # Evaluate model
            valid_loss = evaluate(model, dev_loader, criterion)

            print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f}")
            # Log the evaluation metrics
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "validation_loss": valid_loss,
                })

        # Calculate accuracy on validation set
        val_accuracy = calculate_accuracy(model, dev_loader, src_vocab, tgt_vocab, device)
        print(f"\nValidation Accuracy: {val_accuracy:.4f}")
        wandb.log({
            "val_accuracy": val_accuracy
        })

In [24]:
sweep_config = {
    "method" : "bayes",
    "metric" : {"name": "val_accuracy", "goal": "maximize"},
    "parameters" : {
        "embedding_size" : {"values" : [16, 32, 64, 256, 512, 1024]},
        "num_layers" : {"values" : [1, 2, 3, 4]},
        "hidden_size" : {"values" : [16, 32, 64, 128, 256, 512, 1024, 2048]},
        "cell_type" : {"values" : ["RNN", "LSTM", "GRU"]},
        "dropout" : {"values" : [0.0, 0.2, 0.3, 0.4]},
        "learning_rate" : {"values" : [0.01, 0.001, 0.0005]},
        "batch_size": {"values": [16, 32]}
    }
}

sweep_id = wandb.sweep(sweep_config, project = "da24m021_da6401_assignment3")
wandb.agent(sweep_id, function = sweep_hyperparameters, count = 30)

Create sweep with ID: 6g2ha9nv
Sweep URL: https://wandb.ai/da24m021-indian-institute-of-technology-madras/da24m021_da6401_assignment3/sweeps/6g2ha9nv


wandb: Agent Starting Run: x760x6ri with config:
wandb: 	batch_size: 32
wandb: 	cell_type: GRU
wandb: 	dropout: 0.4
wandb: 	embedding_size: 16
wandb: 	hidden_size: 2048
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 1
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250514_062949-x760x6ri
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sandy-sweep-1
wandb: ⭐️ View project at https://wandb.ai/da24m021-indian-institute-of-technology-madras/da24m021_da6401_assignment3
wandb: 🧹 View sweep at https://wandb.ai/da24m021-indian-institute-of-technology-madras/da24m021_da6401_assignment3/sweeps/6g2ha9nv
wandb: 🚀 View run at https://wandb.ai/da24m021-indian-institute-of-technology-madras/da24m021_da6401_assignment3/runs/x760x6ri
 10%|█         | 1/10 [00:42<06:23, 42.56s/it]

Train Loss: 1.0674 | Valid Loss: 0.9679


 20%|██        | 2/10 [01:24<05:39, 42.44s/it]

Train Loss: 0.3931 | Valid Loss: 0.8412


...

Train Loss: 0.0923 | Valid Loss: 0.9982


100%|██████████| 10/10 [37:31<00:00, 225.17s/it]

Train Loss: 0.0832 | Valid Loss: 1.0626



Validation Accuracy: 0.3903


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:           epoch ▁▂▃▃▄▅▆▆▇█
wandb:      train_loss █▄▂▂▁▁▁▁▁▁
wandb:    val_accuracy ▁
wandb: validation_loss █▂▁▁▁▂▂▂▂▃
wandb: 
wandb: Run summary:
wandb:           epoch 9
wandb:      train_loss 0.08316
wandb:    val_accuracy 0.39032
wandb: validation_loss 1.06259
wandb: 
wandb: 🚀 View run embedding_size_16_num_layers_3_hidden_size_2048                           _cell_type_LSTM_dp_0.2_lr_0.0005_batch_32 at: https://wandb.ai/da24m021-indian-institute-of-technology-madras/da24m021_da6401_assignment3/runs/zammv2zj
wandb: ⭐️ View project at: https://wandb.ai/da24m021-indian-institute-of-technology-madras/da24m021_da6401_assignment3
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250514_172137-zammv2zj/logs
